### MNIST data set learning using TensorFlow

Guide link: https://www.tensorflow.org/get_started/mnist/beginners

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
import tensorflow as tf

# Implementing model

# Input value for TF running session. Would be a flatten image from MNIST dataset. 784 = 28x28 pixel image
# placeholder - a value that we'll input when we ask TensorFlow to run a computation.
x = tf.placeholder(tf.float32, [None, 784]) 

# Weight - zeros paramas: diamensional image vectors | 10-dimensional vectors represent number classes in range of 0-9
W = tf.Variable(tf.zeros([784, 10])) 

b = tf.Variable(tf.zeros([10])) # Bias added to a to output

# Create regression model using values above. 
y = tf.matmul(x, W) + b


In [41]:
# This set will be storing predictions
# Each row is a one-hot 10-dimensional vector indicating which digit class 
# (zero through nine) the corresponding MNIST image belongs to.
y_ = tf.placeholder(tf.float32, [None, 10])


# Cross entropy - the average length of communicating an event from one distribution 
# with the optimal code for another distribution. Ref.: http://colah.github.io/posts/2015-09-Visual-Information/

# In this case, it will help to predict how close TF prediction of a number is to the actual correct answer.
# Loss function indicates how bad the model's prediction was on a single example; 
# we try to minimize that while training across all the examples.
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# Pick optimization algorithm.
# GradientDescentOptimizer defines a rate at which TF is going to descend the cross entropy. 
# We set it to 0.5, but should be adjusted.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Create TF study session. Session is connecting to TF backend written in C++ to do heavy computation.
# InteractiveSession is a convenient way to build cumputation graphs. If you not using InteractiveSession,
# you will have to build 
sess = tf.InteractiveSession()
# Initialize variables for TF
tf.global_variables_initializer().run()

# We are going to run training step 1000 times. Again, it should be adjusted for optimal learning time.
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(50) # Picks 100 random data points from set. 
                                                   # More data TF algorithms use to train, better accuracy is produces.
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys}) # Run training session

In [42]:
# Evaluating model
# Handpick correct predictions from y_ set
# tf.argmax(y,1) is the label our model thinks is most likely for each input, 
# while tf.argmax(y_,1) is the true label.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# Evaluate accuracy on the test data
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Print correct prediction rate 
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9155
